### 残差网络（ResNet）

让我们先思考一个问题：对神经网络模型添加新的层，充分训练后的模型是否只可能更有效地降低训练误差？

理论上，原模型解的空间只是新模型解的空间的子空间。也就是说，如果我们能将新添加的层训练成恒等映射f(x)=x
，新模型和原模型将同样有效。由于新模型可能得出更优的解来拟合训练数据集，因此添加层似乎更容易降低训练误差。然而在实践中，添加过多的层后训练误差往往不降反升。即使利用批量归一化带来的数值稳定性使训练深层模型更加容易，该问题仍然存在。针对这一问题，何恺明等人提出了残差网络（ResNet） [1]。它在2015年的ImageNet图像识别挑战赛夺魁，并深刻影响了后来的深度神经网络的设计。

#### 残差块

让我们聚焦于神经网络局部。如图5.9所示，设输入为x
。假设我们希望学出的理想映射为f(x)，从而作为图5.9上方激活函数的输入。左图虚线框中的部分需要直接拟合出该映射f(x)，而右图虚线框中的部分则需要拟合出有关恒等映射的残差映射f(x)−x。残差映射在实际中往往更容易优化。以本节开头提到的恒等映射作为我们希望学出的理想映射f(x)。我们只需将图5.9中右图虚线框内上方的加权运算（如仿射）的权重和偏差参数学成0，那么f(x)即为恒等映射。实际中，当理想映射f(x)极接近于恒等映射时，残差映射也易于捕捉恒等映射的细微波动。图5.9右图也是ResNet的基础块，即残差块（residual block）。在残差块中，输入可通过跨层的数据线路更快地向前传播。
<img src= "img/residual-block.svg">

ResNet沿用了VGG全3×3卷积层的设计。残差块里首先有2个有相同输出通道数的3×3卷积层。每个卷积层后接一个批量归一化层和ReLU激活函数。然后我们将输入跳过这两个卷积运算后直接加在最后的ReLU激活函数前。这样的设计要求两个卷积层的输出与输入形状一样，从而可以相加。如果想改变通道数，就需要引入一个额外的1×1卷积层来将输入变换成需要的形状后再做相加运算。


#### ResNet模型

ResNet的前两层跟之前介绍的GoogLeNet中的一样：在输出通道数为64、步幅为2的7×7卷积层后接步幅为2的3×3的最大池化层。不同之处在于ResNet每个卷积层后增加的批量归一化层。

GoogLeNet在后面接了4个由Inception块组成的模块。ResNet则使用4个由残差块组成的模块，每个模块使用若干个同样输出通道数的残差块。第一个模块的通道数同输入通道数一致。由于之前已经使用了步幅为2的最大池化层，所以无须减小高和宽。之后的每个模块在第一个残差块里将上一个模块的通道数翻倍，并将高和宽减半。

#### 实现

In [50]:
import numpy as np
import keras
from keras.optimizers import SGD
from keras.models import Model
from keras.layers import Input, GlobalAveragePooling2D,Activation, Dense
from keras.layers import Conv2D, MaxPool2D, BatchNormalization


# 残差块
def residual(input, num_channels, kernel_size=3, stride=1, padding="same", use_1x1conv=False):
    x = input 
    
    y = Conv2D(filters=num_channels, kernel_size=(kernel_size, kernel_size), 
               strides=(stride,stride), padding = padding)(input)
    y = BatchNormalization(axis=3)(y)
    y = Activation('relu')(y)
    
    # 第二个3x3卷积
    Conv2D(filters=num_channels, kernel_size=(kernel_size, kernel_size), 
               strides=(stride,stride), padding = padding)(y)
    y = BatchNormalization(axis=3)(y)
    
    # 如果使用了 1 x 1的卷积层，那么就要让input卷积一下，改变input的通道数
    if use_1x1conv:
        x = Conv2D(filters=num_channels, kernel_size=(1, 1), 
               strides=(stride,stride), padding = padding)(x)
        
       # 相加
    z = keras.layers.add([x, y])
    
    return Activation('relu')(z)

# ResNet模型
# ResNet的前两层跟之前介绍的GoogLeNet中的一样：在输出通道数为64、步幅为2的7×7卷积层后接步幅为2的3×3的最大池化层。
# 不同之处在于ResNet每个卷积层后增加的批量归一化层。



# 输入
input_img = Input(shape=(224, 224, 1))
# x = residual(input_img, 3)
# 通道数增加一倍，但是高宽减半  (6, 3, 3)
# x = residual(input_img, num_channels=6, use_1x1conv=True, stride=2)

x = Conv2D(filters=64, kernel_size=(7, 7), 
               strides=(2,2), padding = "same")(input_img)
x = BatchNormalization(axis=3)(x)
x = Activation('relu')(x)
x = MaxPool2D(pool_size=(3,3), strides=(2,2), padding='same')(x)

# GoogLeNet在后面接了4个由Inception块组成的模块。
# ResNet则使用4个由残差块组成的模块，每个模块使用若干个同样输出通道数的残差块。
# 第一个模块的通道数同输入通道数一致。由于之前已经使用了步幅为2的最大池化层，所以无须减小高和宽。
# 之后的每个模块在第一个残差块里将上一个模块的通道数翻倍，并将高和宽减半。

# 下面我们来实现这个模块。注意，这里对第一个模块做了特别处理。
def resnet_block(x, num_channels, num_residuals, first_block=False):
    for i in range(num_residuals):
        if i == 0 and not first_block:
            x = residual(input=x, num_channels=num_channels, use_1x1conv=True, stride=2)
        else:
            x = residual(input=x, num_channels=num_channels)
    return x

# 接着我们为ResNet加入所有残差块。这里每个模块使用两个残差块。
x = resnet_block(x, 64, 2, first_block=True)
x = resnet_block(x, 128, 2)
x = resnet_block(x, 256, 2)
x = resnet_block(x, 512, 2)

# 最后，与GoogLeNet一样，加入全局平均池化层后接上全连接层输出。
x = GlobalAveragePooling2D()(x)

# 最后全连接成 class 类别 10
x = Dense(10, activation='softmax')(x)


model = Model(inputs=input_img, outputs=x)



# conv5 output shape:      (1, 64, 112, 112)
# batchnorm4 output shape:         (1, 64, 112, 112)
# relu0 output shape:      (1, 64, 112, 112)
# pool0 output shape:      (1, 64, 56, 56)
# sequential1 output shape:        (1, 64, 56, 56)
# sequential2 output shape:        (1, 128, 28, 28)
# sequential3 output shape:        (1, 256, 14, 14)
# sequential4 output shape:        (1, 512, 7, 7)
# pool1 output shape:      (1, 512, 1, 1)
# dense0 output shape:     (1, 10)



In [51]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_39 (InputLayer)           (None, 224, 224, 1)  0                                            
__________________________________________________________________________________________________
conv2d_67 (Conv2D)              (None, 112, 112, 64) 3200        input_39[0][0]                   
__________________________________________________________________________________________________
batch_normalization_49 (BatchNo (None, 112, 112, 64) 256         conv2d_67[0][0]                  
__________________________________________________________________________________________________
activation_45 (Activation)      (None, 112, 112, 64) 0           batch_normalization_49[0][0]     
__________________________________________________________________________________________________
max_poolin

#### 小结

    残差块通过跨层的数据通道从而能够训练出有效的深度神经网络。
    ResNet深刻影响了后来的深度神经网络的设计。

#### 思考

    参考ResNet论文的表1来实现不同版本的ResNet [1]。
    对于比较深的网络， ResNet论文中介绍了一个“瓶颈”架构来降低模型复杂度。尝试实现它 [1]。
    在ResNet的后续版本里，作者将残差块里的“卷积、批量归一化和激活”结构改成了“批量归一化、激活和卷积”，实现这个改进（[2]，图1）。

